In [1]:
# PERIOD:  extract data in period

# 行求和：https://blog.csdn.net/weixin_43629813/article/details/119894001
import sys
import pandas as pd

COST=82591.14
ICOST=1751.20
ACOST=COST-ICOST

acpsid_loc='cpsid-A.txt'
icpsid_loc='cpsid-I.txt'
test_cpsid_loc='cpsid-test.txt'

# # 1、get cpsid 
# acpsid=[]
# atxt=open('cpsid-A.txt','r').readlines()
# for aline in atxt:
#     aline=aline.replace('\n','')
#     acpsid.append(aline)


# icpsid=[]
# itxt=open('cpsid-I.txt','r').readlines()
# for iline in itxt:
#     iline=iline.replace('\n','')
#     icpsid.append(iline)

def get_cpsid(cpsid_txt):
    cpsid=[]
    txt=open(cpsid_txt,'r').readlines()
    for line in txt:
        line=line.replace('\n','')
        cpsid.append(float(line))
    return cpsid    

acpsid=get_cpsid(acpsid_loc)
icpsid=get_cpsid(icpsid_loc)
test_cpsid=get_cpsid(test_cpsid_loc)

# 2、read data
data=pd.read_excel('data.xlsx')
#data=data['f1'].astype(float)
data['日期']=data['日期'].str.split(" ",expand=True)[0]
data=data.replace('-',0)
data['f3']=data['f3'].astype(float)
data['f7']=data['f7'].astype(float)
data['f15']=data['f15'].astype(float)
data['f30']=data['f30'].astype(float)
data['f60']=data['f60'].astype(float)
data['f90']=data['f90'].astype(float)
data['f180']=data['f180'].astype(float)
data['c1']=data['c1'].astype(float)
data['至今付费']=data['至今付费'].astype(float)



target_data_a=data[data['广告id'].isin(acpsid)]
target_data_i=data[data['广告id'].isin(icpsid)]
target_data_test=data[data['广告id'].isin(test_cpsid)]

df=pd.DataFrame(columns=("消耗","激活","c1","首日付费","至今付费","有效数"),index=['Android','IOS'])
df.loc['Android']=[ACOST,target_data_a['激活'].sum(),target_data_a['c1'].sum(),target_data_a['f1'].sum(),target_data_a['至今付费'].sum(),target_data_a['有效激活设备数'].sum()]
df.loc['IOS']=[ICOST,target_data_i['激活'].sum(),target_data_i['c1'].sum(),target_data_i['f1'].sum(),target_data_i['至今付费'].sum(),target_data_i['有效激活设备数'].sum()]
df.loc['ALL']=df.apply(lambda x:x.sum())
df["激活成本"]=df["消耗"]/df['激活']
df["首日ROI"]=df["首日付费"]/df['消耗']*100
df["至今ROI"]=df["至今付费"]/df['消耗']*100
df["有效率"]=df["有效数"]/df['激活']*100


print(df)



writer=pd.ExcelWriter('data_extract_out.xlsx')
df.to_excel(writer,"ALL")
target_data_a.to_excel(writer,'安卓',index=False)
target_data_i.to_excel(writer,'IOS',index=False)
target_data_test.to_excel(writer,'test',index=False)
writer.save()
print("done")



               消耗      激活     c1     首日付费      至今付费     有效数       激活成本  \
Android  80839.94  5157.0  750.0  35525.0  586585.0  2148.0  15.675769   
IOS        1751.2   269.0   37.0  10995.0   83075.0   114.0   6.510037   
ALL      82591.14  5426.0  787.0  46520.0  669660.0  2262.0  15.221367   

              首日ROI        至今ROI        有效率  
Android   43.944862   725.612859  41.652123  
IOS      627.855185  4743.889904  42.379182  
ALL       56.325654   810.813363  41.688168  
done


In [2]:
# daily func
# extract daily data
# most paid cpsid
import sys
import pandas as pd
import numpy as np

COST=1458
ICOST=0
ACOST=COST-ICOST

data_loc='data-daily.xlsx'
ATYPE='Android'
acpsid_loc='cpsid-A.txt'
ITYPE='IOS'
icpsid_loc='cpsid-I.txt'



# 1、get cpsid 
def get_cpsid(cpsid_txt):
    cpsid=[]
    txt=open(cpsid_txt,'r').readlines()
    for line in txt:
        line=line.replace('\n','')
        cpsid.append(float(line))
    return cpsid

# 2、extract data
def get_data(data_loc):
    data=pd.read_excel(data_loc)
    data=data.replace('-',0)
    data['付费']=data['付费'].astype(float)
    return data

def extract_data(cpsid,data,COST,TYPE):
    #df_list=[]
    data=data[data['cpsid'].isin(cpsid)]
    data_group=data.groupby('cpsid').agg({'时间':max})
    data_group=data_group.astype(object)
    new_data=data_group.merge(data,on=['cpsid','时间'])
    df.loc[TYPE]=[COST,new_data["激活"].sum(),new_data["注册"].sum(),new_data["支付账号"].sum(),new_data["付费"].sum(),new_data["创角"].sum(),new_data["有效激活设备数"].sum()]
    #print(df.loc[TYPE])
    return [new_data,data]



acpsid=get_cpsid(acpsid_loc)
icpsid=get_cpsid(icpsid_loc)

data=get_data(data_loc)
df=pd.DataFrame(columns=("消耗","激活","注册","支付账号","付费","创角","有效激活设备数"),index=['Android','IOS'])
aExtract=extract_data(acpsid,data,ACOST,ATYPE)
iExtract=extract_data(icpsid,data,ICOST,ITYPE)
df.loc['ALL']=df.apply(lambda x:x.sum())
print(df)


writer=pd.ExcelWriter('data_daily_extract.xlsx')

df.to_excel(writer,"Sum")
aExtract[0].to_excel(writer,'安卓-part',index=False)
iExtract[0].to_excel(writer,'IOS-part',index=False)
aExtract[1].to_excel(writer,'安卓-ALL',index=False)
iExtract[1].to_excel(writer,'IOS-ALL',index=False)
writer.save()

print("done")



             消耗    激活   注册 支付账号    付费   创角 有效激活设备数
Android    1458    10    5    2  12.0    5       4
IOS           0     0    0    0   0.0    0       0
ALL      1458.0  10.0  5.0  2.0  12.0  5.0     4.0
done


In [7]:
# daily most paid data

import pandas as pd
import numpy as np

def get_data(data_loc):
    data=pd.read_excel(data_loc)
    data=data.replace('-',0)
    data['付费']=data['付费'].astype(float)
    return data

csList=['210','211','212','213','871','872','873']
data_loc='data-daily.xlsx'
threshold=50

data=get_data(data_loc)
data_group=data.groupby('cpsid').agg({'时间':max})
data_group=data_group.astype(object)
data=data_group.merge(data,on=['cpsid','时间'])


data['cpsid']=data['cpsid'].astype(str)

data=data[data['cpsid'].str[:3].isin(csList)]

data=data[data['付费']>threshold]
data.sort_values(by=['付费'],inplace=True,ascending=False)
print(data)


writer=pd.ExcelWriter('data_daily_most_paid.xlsx')
data.to_excel(writer,'most-paid',index=False)
writer.save()

       cpsid                   时间  渠道 归因方法  激活  注册  创角  有效激活设备数  支付账号      付费  \
113  2110091  2022-05-28 23:54:06  yc   TD   1   1   3        1     1  3148.0   
46   2101431  2022-05-28 23:54:06  yc   TD   8   6   8        4     3   338.0   
88   2102635  2022-05-28 23:54:06  yc   TD  25  23  26       15     4   282.0   
162  2130087  2022-05-28 23:54:06  yc   TD  19  12  10        6     2   134.0   
82   2102248  2022-05-28 23:54:06  yc   TD   3   2   1        1     1    98.0   
87   2102633  2022-05-28 23:54:06  yc   TD  24  20  26       13     6    66.0   

     三百以上人次  一百以上  五十以上  十元以上  一元以上  
113       5     1     0     1     0  
46        0     1     1     2     2  
88        0     0     1     6     7  
162       0     0     1     2     1  
82        0     0     1     1     0  
87        0     0     0     1     6  


In [7]:
# PERIOD:  extract data in period

# 行求和：https://blog.csdn.net/weixin_43629813/article/details/119894001
import sys
import pandas as pd

COST=82591.14
ICOST=1751.20
ACOST=COST-ICOST

acpsid_loc='cpsid-A.txt'
icpsid_loc='cpsid-I.txt'
test_cpsid_loc='cpsid-test.txt'

# # 1、get cpsid 
# acpsid=[]
# atxt=open('cpsid-A.txt','r').readlines()
# for aline in atxt:
#     aline=aline.replace('\n','')
#     acpsid.append(aline)


# icpsid=[]
# itxt=open('cpsid-I.txt','r').readlines()
# for iline in itxt:
#     iline=iline.replace('\n','')
#     icpsid.append(iline)

def get_cpsid(cpsid_txt):
    cpsid=[]
    txt=open(cpsid_txt,'r').readlines()
    for line in txt:
        line=line.replace('\n','')
        cpsid.append(float(line))
    return cpsid    

acpsid=get_cpsid(acpsid_loc)
icpsid=get_cpsid(icpsid_loc)
test_cpsid=get_cpsid(test_cpsid_loc)

# 2、read data
data=pd.read_excel('data1.xlsx')




target_data_a=data[data['cpsid'].isin(acpsid)]
target_data_i=data[data['cpsid'].isin(icpsid)]
target_data_test=data[data['cpsid'].isin(test_cpsid)]





writer=pd.ExcelWriter('data_extract.xlsx')

target_data_a.to_excel(writer,'安卓',index=False)
target_data_i.to_excel(writer,'IOS',index=False)
target_data_test.to_excel(writer,'test',index=False)
writer.save()
print("done")



done
